# PyRadio Antenna Module Examples

This notebook demonstrates how to use the antenna module in PyRadio to model different types of antennas and calculate properties like gain and polarization loss.

In [1]:
# Import the antenna module and units
from spacelink.antenna import Antenna, FixedGain, Dish, polarization_loss
from spacelink.units import Q_, GHz, m, db, km
from spacelink.path import free_space_path_loss

In [2]:
print(free_space_path_loss(384400 * km, 7.2 * GHz).to('dB'))

-221.2901007304389 decibel


## Creating Different Types of Antennas

PyRadio supports different antenna types including:
- `FixedGain`: Simple antenna with constant gain
- `Dish`: Parabolic dish antenna with gain based on diameter, efficiency, and frequency

In [3]:
# Create a fixed gain antenna (like a patch or dipole)
# This has a gain of 2.15 dB and a circular polarization
patch_antenna = FixedGain(2.15, axial_ratio=0.0)

# Create a dish antenna with 3.7m diameter and 65% efficiency
# This has a slightly elliptical polarization (axial ratio = 2.0 dB)
dish_antenna = Dish(Q_(3.7, 'm'), efficiency=0.65, axial_ratio=2.0)


## Calculating Antenna Gain

The gain of a `FixedGain` antenna is constant regardless of frequency, while the gain of a `Dish` antenna depends on frequency.

In [4]:
# Define frequencies for testing
s_band = Q_(2.2, 'GHz')
x_band = Q_(8.4, 'GHz')
ka_band = Q_(32.0, 'GHz')

# Calculate and display the gain of the fixed gain antenna
print(f"Fixed gain antenna at any frequency: {patch_antenna.gain(s_band):.2f} dB")

# Calculate and display the gain of the dish antenna at different frequencies
print(f"Dish antenna at S-band ({s_band}): {dish_antenna.gain(s_band):.2f} dB")
print(f"Dish antenna at X-band ({x_band}): {dish_antenna.gain(x_band):.2f} dB")
print(f"Dish antenna at Ka-band ({ka_band}): {dish_antenna.gain(ka_band):.2f} dB")

Fixed gain antenna at any frequency: 2.15 dB
Dish antenna at S-band (2.2 gigahertz): 36.75 dB
Dish antenna at X-band (8.4 gigahertz): 48.39 dB
Dish antenna at Ka-band (32.0 gigahertz): 60.00 dB


## Polarization Loss

Polarization loss occurs when the polarization of the transmitting and receiving antennas don't match perfectly. The `polarization_loss` function calculates this loss based on the axial ratios of both antennas.

In [6]:
# Calculate polarization loss between different antenna pairs
loss1 = polarization_loss(patch_antenna.axial_ratio, dish_antenna.axial_ratio)

print(f"Polarization loss between circular and elliptical: {loss1:.2f} dB")

Polarization loss between circular and elliptical: -0.11 dB


## Table of Polarization Loss Values

Below is a table of polarization loss values (in dB) for different combinations of axial ratios.

In [7]:
import pandas as pd
import numpy as np

# Define axial ratio values
ar_values = [0.5, 1.0, 2.0, 3.0, 4.0]

# Create a matrix of polarization loss values
loss_matrix = np.zeros((len(ar_values), len(ar_values)))
for i, tx_ar in enumerate(ar_values):
    for j, rx_ar in enumerate(ar_values):
        loss_matrix[i, j] = polarization_loss(tx_ar, rx_ar)

# Create and display the table
ar_table = pd.DataFrame(loss_matrix, index=ar_values, columns=ar_values)
ar_table.index.name = 'Tx AR (dB)'
ar_table.columns.name = 'Rx AR (dB)'
ar_table

Rx AR (dB),0.5,1.0,2.0,3.0,4.0
Tx AR (dB),,,,,
0.5,-0.014383,-0.035911,-0.121318,-0.261240,-0.452296
1.0,-0.035911,-0.057438,-0.142845,-0.282768,-0.473824
2.0,-0.121318,-0.142845,-0.228252,-0.368175,-0.559231
3.0,-0.261240,-0.282768,-0.368175,-0.508097,-0.699153
4.0,-0.452296,-0.473824,-0.559231,-0.699153,-0.890209


## Creating a Custom Antenna Implementation

You can create your own antenna type by subclassing the `Antenna` abstract base class and implementing the required `gain` method.

In [8]:
# Example of creating a custom antenna implementation
class YagiAntenna(Antenna):
    """
    A simple model of a Yagi-Uda antenna with gain that varies by frequency.
    """
    def __init__(self, elements: int, center_freq: 'Quantity', axial_ratio: float = 40.0):
        super().__init__(axial_ratio)  # Yagis are typically linearly polarized
        self.elements = elements
        self.center_freq = center_freq
    
    def gain(self, frequency: 'Quantity') -> float:
        """
        Calculate the gain based on elements and frequency relative to design frequency.
        This is a simplified model.
        """
        # Base gain based on number of elements (a simple approximation)
        base_gain = 3.0 + (self.elements - 3) * 2.0
        
        # Frequency roll-off (gain drops as we move away from center frequency)
        freq_ratio = (frequency / self.center_freq).to_base_units().magnitude
        freq_factor = 1 - min(0.5, abs(freq_ratio - 1) * 2.0)
        
        return base_gain * freq_factor

# Create and test our custom antenna
yagi = YagiAntenna(elements=8, center_freq=Q_(435, 'MHz'))
test_freqs = [Q_(400, 'MHz'), Q_(435, 'MHz'), Q_(470, 'MHz')]

for freq in test_freqs:
    print(f"Yagi gain at {freq}: {yagi.gain(freq):.2f} dB")

Yagi gain at 400 megahertz: 10.91 dB
Yagi gain at 435 megahertz: 13.00 dB
Yagi gain at 470 megahertz: 10.91 dB
